# Statistics


**IDEAS**
- SMETANA
    - Find likelyhood that a community member from family X is dependent on a community member in family Y.
    - Find likelyhood that a community member from family X is dependent on a compound A.

- SteadierCOM
    - No statistics?

In [111]:
import math

import pandas as pd
from molmass import Formula
import reframed

In [112]:
import sys
sys.path.append("../functions/")
import general_functions as general_func

import colors_MAGs
import scipy.stats as stats

In [113]:
import numpy as np

### Load universal model

In [114]:
model_uni = reframed.load_cbmodel("/Users/idunmariaburgos/universal_model_extension/output/universe_bacteria.xml")

### Load data 

In [115]:
all_mags_paper = general_func.read_allmags_data()

In [116]:
SC1_C = pd.read_csv("../output/steadiercom_sample_0.1.3/results/results_99_SC1_C.tsv",sep="\t")
SC2_C = pd.read_csv("../output/steadiercom_sample_0.1.3/results/results_99_SC2_C.tsv",sep="\t")
#SC1_X = pd.read_csv("../output/steadiercom_sample_0.1.3/results/results_99_SC1_X.tsv",sep="\t")

steadier_sample = pd.concat([SC1_C,SC2_C])#,SC1_X])
steadier_sample = steadier_sample[(steadier_sample.frequency>0.1) & (steadier_sample.rate>1e-6)]

In [117]:
steadier_sample_cross = steadier_sample[(steadier_sample.donor!="environment") & (steadier_sample.receiver!="environment") ].copy()

### Prepare to process data

In [118]:
chebi_lut, chebi_interesting, chebi_colors_ser = colors_MAGs.chebi_rxn_color_func(rxn_based=False)

**Change names of family for readability and better grouping**

In [119]:
all_mags_paper_reduced = all_mags_paper[~(all_mags_paper.Substrate=="Xylan")].copy()

all_mags_paper_reduced[all_mags_paper_reduced["new_coverage"]>10][["Source","Substrate","Family","new_coverage"]].sort_values(["Source","Substrate"])

all_mags_paper_reduced["Family"] = all_mags_paper.apply(lambda row: "f_"+row.Family,axis=1)

total_members_family = all_mags_paper_reduced.groupby("Family").count()["Source"].to_dict()
all_mags_paper_reduced["Family"] = all_mags_paper_reduced.apply(lambda row: row.Family if total_members_family[row.Family]>1 else "Other",axis=1)


In [120]:
def mag2family(all_mags_paper):  
    
    family_groups = all_mags_paper.groupby("Family").groups
    mag2family_dict = {mag:family for family,mags in family_groups.items() for mag in mags}
    
    return family_groups,mag2family_dict

**Create dictionaries to translate components into larger groups - MAG-> family, compound-> super_class**

In [121]:
MAGs_steady_com = set(list(steadier_sample_cross.donor.values)+list(steadier_sample_cross.receiver.values))

MAG2sour_sub_id = pd.read_csv("../output/MAG2community_id.tsv",sep="\t",header=None)
MAG2sour_sub_id.columns=["MAG","community_id"]

met2superclass_dict = pd.read_csv("../output/met_chebi_class.tsv",sep="\t",index_col=0)["self defined super class"].to_dict()

family_groups,mag2family_dict = mag2family(all_mags_paper_reduced)

In [122]:
def family_donor(row):
    if row.donor=="environment":
        return "environment"
    else:
        return mag2family_dict[row.donor]


def family_receiver(row):
    if row.receiver=="environment":
        return "environment"
    else:
        return mag2family_dict[row.receiver]


In [123]:
def met2metname(met):
    met_name = model_uni.metabolites[met].name
    return met_name

### Process data

**All compounds**

In [124]:
steadier_sample.loc[:,"family_donor"] = steadier_sample.apply(family_donor,axis=1).copy()
steadier_sample.loc[:,"family_receiver"] = steadier_sample.apply(family_receiver,axis=1).copy()
steadier_sample = steadier_sample[steadier_sample.compound.isin(met2superclass_dict.keys())].copy()
steadier_sample.loc[:,"super_class"] = steadier_sample.apply(lambda x: met2superclass_dict[x.compound],axis=1)

compounds = steadier_sample["compound"].map(met2metname)
steadier_sample.drop("compound",axis=1,inplace=True)
steadier_sample.loc[:,"compound"] = compounds


**Compounds cross-fed**

In [125]:
steadier_sample_cross.loc[:,"family_donor"] = steadier_sample_cross.apply(family_donor,axis=1).copy()
steadier_sample_cross.loc[:,"family_receiver"] = steadier_sample_cross.apply(family_receiver,axis=1).copy()
steadier_sample_cross = steadier_sample_cross[steadier_sample_cross.compound.isin(met2superclass_dict.keys())].copy()
steadier_sample_cross.loc[:,"super_class"] = steadier_sample_cross.apply(lambda x: met2superclass_dict[x.compound],axis=1)

compounds = steadier_sample_cross["compound"].map(met2metname)
steadier_sample_cross.drop("compound",axis=1,inplace=True)
steadier_sample_cross.loc[:,"compound"] = compounds

# FILTERS OUT UNINTERESTING COMPOUNDS
steadier_sample_cross = steadier_sample_cross[steadier_sample_cross.super_class.isin(chebi_lut.keys())].copy()

steadier_sample_cross["mass_rate*frequency"]=steadier_sample_cross["mass_rate"]*steadier_sample_cross["frequency"]

In [126]:
steadier_sample_cross

,donor,receiver,mass_rate,rate,frequency,community,medium,family_donor,family_receiver,super_class,compound,mass_rate*frequency
13,CH15-bin.0,CH15-bin.7,4.921516e-03,0.027318,0.27,CD_P,SC1_C,f_Treponemataceae,f_Sphaerochaetaceae,simple sugars,D-Glucose,1.328809e-03
33,CH13-bin.11,CH13-bin.4,2.822338e-03,0.008245,0.11,CD_A,SC1_C,f_Desulfovibrionaceae,f_Bacteroidaceae,oligosaccharides,Cellobiose,3.104572e-04
36,CH13-bin.14,CH13-bin.25,2.705991e-03,0.015020,0.20,CD_A,SC1_C,f_Lachnospiraceae,f_Sphaerochaetaceae,simple sugars,D-Glucose,5.411983e-04
39,CH15-bin.7,CH15-bin.8,2.435377e-03,0.013518,0.12,CD_P,SC1_C,f_Sphaerochaetaceae,f_Desulfovibrionaceae,simple sugars,D-Glucose,2.922452e-04
41,CH15-bin.23,CH15-bin.8,2.385105e-03,0.013239,0.17,CD_P,SC1_C,Other,f_Desulfovibrionaceae,simple sugars,D-Glucose,4.054678e-04
...,...,...,...,...,...,...,...,...,...,...,...,...
1554,CH7-bin.8,CH7-bin.1,3.864340e-08,0.000001,0.69,CM_P,SC2_C,f_Sphaerochaetaceae,Other,gases,Hydrogen sulfide,2.666394e-08
1555,CH7-bin.15,CH7-bin.18,3.769972e-08,0.000001,0.24,CM_P,SC2_C,f_Sphaerochaetaceae,f_Desulfobulbaceae,gases,Hydrogen sulfide,9.047933e-09
1558,CH7-bin.9,CH7-bin.18,3.640885e-08,0.000001,0.24,CM_P,SC2_C,f_Lentimicrobiaceae,f_Desulfobulbaceae,gases,Hydrogen sulfide,8.738125e-09
1560,CH8-bin.14,CH8-bin.9,3.509299e-08,0.000001,0.66,CM_A,SC2_C,f_CAG-74,Other,gases,Hydrogen sulfide,2.316138e-08


### Overview of groups

In [127]:
pd.Series({family:len(mags) for family,mags in family_groups.items()})

Other                    15
f_Bacteroidaceae          4
f_CAG-74                  2
f_Desulfobulbaceae        2
f_Desulfovibrionaceae    11
f_Dysgonomonadaceae       4
f_Fibrobacteraceae        2
f_Lachnospiraceae         5
f_Lentimicrobiaceae       2
f_Sphaerochaetaceae       6
f_Treponemataceae         4
f_Verruco-01              3
dtype: int64

In [128]:
all_mags_paper_reduced[all_mags_paper_reduced["new_coverage"]>10][["Source","Substrate","Family","Genus","new_coverage"]].sort_values(["Source","Substrate"])

,Source,Substrate,Family,Genus,new_coverage
MAG,,,,,
CH13-bin.12,Compost_Digestate,Avicel,f_Fibrobacteraceae,Fibro-01,36.526676
CH13-bin.11,Compost_Digestate,Avicel,f_Desulfovibrionaceae,Desulfovibrio,10.155790
CH13-bin.4,Compost_Digestate,Avicel,f_Bacteroidaceae,Bacteroides,10.043970
CH15-bin.0,Compost_Digestate,PASC,f_Treponemataceae,DUOS01,23.919040
CH15-bin.23,Compost_Digestate,PASC,Other,DTFZ01,10.319866
CH8-bin.22,Cow_Manure,Avicel,f_Fibrobacteraceae,Fibro-01,42.725525
CH7-bin.23,Cow_Manure,PASC,f_Desulfovibrionaceae,Halodesulfovibrio,38.075882
CH1-bin.6,Marshland,PASC,f_Lachnospiraceae,Lacrimispora,45.242630
CH1-bin.9,Marshland,PASC,f_Desulfovibrionaceae,Solidesulfovibrio,31.170460


### Functions for statistics

In [129]:
def find_non_dependent(row,metric):
    return len(family_groups[row.name[0]]) - row[metric] #row.flux_mg


def statistics_adjustments(statistics_df):
    
    statistics_df = statistics_df.sort_values(by="p_value").copy()
    statistics_df["i"] = statistics_df["p_value"].rank(method="max")
    statistics_df["p_value_benjamini_h"] = statistics_df.apply(lambda row: min(row.p_value*statistics_df.shape[0]/row.i,1),axis=1)
    statistics_df.sort_index(inplace=True)
    return statistics_df


def statistics_function(steadier_sample_cross,dependent_variable,independent_variable,metric="flux_mg",metric_thresh=1e-6,pvalue_thresh=0.1):

    # Get average of each family according to each possible value of the independent variable
    # dependent_variable,dependent_variable.split("_")[1] here it decides if it is in the receiver or in the donor (dependent_variable.split("_")[1]) and groups by the [family_receiver,receiver,compound] and takes the mean of this
    steadiercom_crossfeeding_donor = steadier_sample_cross.loc[:,[dependent_variable,dependent_variable.split("_")[1],independent_variable,metric]].groupby([dependent_variable,dependent_variable.split("_")[1],independent_variable]).mean().copy()
    dependent = steadiercom_crossfeeding_donor[steadiercom_crossfeeding_donor[metric]>metric_thresh].reset_index().groupby([dependent_variable,independent_variable]).count().copy()
    not_dependent = dependent.apply(find_non_dependent,metric=metric,axis=1)

    # Add data for the missing values
    all_categories =set(not_dependent.index.get_level_values(1))

    for family in dependent.index.get_level_values(0):
        for category in all_categories-set(not_dependent.xs(family).index):
            not_dependent[(family,category)]=len(family_groups[family]) 
            
    concat_df = pd.concat({"dependent":dependent[metric],"not_dependent":not_dependent},axis=1).fillna(0)


    statistics = {}
    for independent_var in set(concat_df.index.get_level_values(1)):
        # Get the sub_df for the super class
        concat_df_sub = concat_df.xs(independent_var,level=1).copy()

        statistics[independent_var] = {}

        # For each row (each family)
        for i,row in concat_df_sub.iterrows():
            
            # Get the data for all other family
            other = concat_df_sub.loc[concat_df_sub.index[concat_df_sub.index!=i],:]
            
            data = pd.DataFrame({i:row,"other":other.sum()}).transpose().to_numpy()
            statistics[independent_var][(i,"data")]= data
            
            
            # Get odds ratio
            odds_ratio_num = data[0][0]
            odds_ratio_den = data[0][0] + data[0][1]
            other_num = data[1][0]
            other_den = data[1][0] + data[1][1]
            
            if odds_ratio_den == 0 or other_den == 0 or other_num==0:
                odds_ratio = math.inf
            else:
                odds_ratio = (odds_ratio_num / odds_ratio_den) / (other_num / other_den)
            statistics[independent_var][(i, "odds_ratio")] = odds_ratio
            
            
            # Calculate the Barnard exact statistical 
            p_value = stats.barnard_exact(pd.DataFrame({i:row,"other":other.sum()}).transpose().to_numpy(),alternative="greater")
            statistics[independent_var][(i,"p_value")]= p_value.pvalue
            

    statistics_df = pd.DataFrame(statistics)
    

    category_values = statistics_df.xs('p_value', level=1)
    

    values = {}
    for family,independent_var in category_values[category_values[category_values<pvalue_thresh].notnull()].stack().index:
        values[family,independent_var]= {"p_value":statistics_df.loc[(family,"p_value"),independent_var],"odds_ratio":statistics_df.loc[(family,"odds_ratio"),independent_var],"data":statistics_df.loc[(family,"data"),independent_var],"# interaction":concat_df.loc[(family,independent_var),"dependent"],"# no interaction":concat_df.loc[(family,independent_var),"not_dependent"]}

    significant = pd.DataFrame(values).transpose()
    if significant.empty==False:
        significant.index.names = (dependent_variable,independent_variable)

    return significant

## Likelyhood that family X is receiving compound A

In [130]:
statistics_df = statistics_function(steadier_sample_cross,"family_receiver","compound",metric="rate",metric_thresh=1e-6,pvalue_thresh=1)
statistics_df = statistics_adjustments(statistics_df)
statistics_df.shape

(134, 7)

In [131]:
statistics_df[(statistics_df.p_value_benjamini_h<0.05) & (statistics_df["odds_ratio"]>4.95)]

,,p_value,odds_ratio,data,# interaction,# no interaction,i,p_value_benjamini_h
family_receiver,compound,,,,,,,
f_Lachnospiraceae,Thymine C5H6N2O2,0.000199,5.0,"[[5.0, 0.0], [11.0, 44.0]]",5.0,0,1.0,0.026658


In [132]:
statistics_df = statistics_function(steadier_sample_cross,"family_receiver","super_class",metric="rate",metric_thresh=1e-6,pvalue_thresh=1)
statistics_df = statistics_adjustments(statistics_df)
statistics_df.shape

(43, 7)

In [133]:
statistics_df[(statistics_df.p_value_benjamini_h<0.1) & (statistics_df["odds_ratio"]>4.95)]

,,p_value,odds_ratio,data,# interaction,# no interaction,i,p_value_benjamini_h
family_receiver,super_class,,,,,,,


## Likelyhood that family X is donating a compound A to another family

In [134]:
statistics_df = statistics_function(steadier_sample_cross,"family_donor","compound",metric="rate",metric_thresh=1e-6,pvalue_thresh=1)
statistics_df = statistics_adjustments(statistics_df)
statistics_df.shape

(115, 7)

In [135]:
statistics_df[(statistics_df.p_value_benjamini_h<0.05) & (statistics_df["odds_ratio"]>4.95)]

,,p_value,odds_ratio,data,# interaction,# no interaction,i,p_value_benjamini_h
family_donor,compound,,,,,,,


In [136]:
statistics_df = statistics_function(steadier_sample_cross,"family_donor","super_class",metric="rate",metric_thresh=1e-6,pvalue_thresh=1)
statistics_df = statistics_adjustments(statistics_df)
statistics_df.shape

(48, 7)

In [137]:
statistics_df[(statistics_df.p_value_benjamini_h<0.1) & (statistics_df["odds_ratio"]>4.95)]

,,p_value,odds_ratio,data,# interaction,# no interaction,i,p_value_benjamini_h
family_donor,super_class,,,,,,,


## Likelyhood that family X is receiving from family Y

In [138]:
statistics_df = statistics_function(steadier_sample_cross,"family_receiver","family_donor",metric="rate",metric_thresh=1e-6,pvalue_thresh=1)
statistics_df = statistics_adjustments(statistics_df)
statistics_df.shape

(72, 7)

In [139]:
statistics_df[(statistics_df.p_value_benjamini_h<0.05) & (statistics_df["odds_ratio"]>4.95)]

,,p_value,odds_ratio,data,# interaction,# no interaction,i,p_value_benjamini_h
family_receiver,family_donor,,,,,,,


### All communities - Relative abundance (above 10 %)

In [140]:
total_members = steadier_sample_cross.groupby("family_donor").nunique()["donor"]

In [141]:
def abundance_statistic(abundance_communities,receiver_or_donor="receiver"):
    
    
    table_abundance_rec_don_dict = {}
    table_not_abundant_rec_don_dict = {}

    for compound in abundance_communities[abundance_communities.receiver_abundance_10].compound.unique():
        if receiver_or_donor=="receiver":
            abund_rec_don = len(abundance_communities[(abundance_communities.receiver_abundance_10) & (abundance_communities.compound==compound)].receiver.unique())
            not_abund_rec_don = len(abundance_communities[(~abundance_communities.receiver_abundance_10) & (abundance_communities.compound==compound)].receiver.unique())
        elif receiver_or_donor=="donor":
            abund_rec_don = len(abundance_communities[(abundance_communities.donor_abundance_10) & (abundance_communities.compound==compound)].donor.unique())
            not_abund_rec_don = len(abundance_communities[(~abundance_communities.donor_abundance_10) & (abundance_communities.compound==compound)].donor.unique())
        
        
        abund_not_rec_don = len(high_abundance) - abund_rec_don
        not_abund_not_rec_don = len(low_abunance) - not_abund_rec_don


        table = [[abund_rec_don,not_abund_rec_don],[abund_not_rec_don,not_abund_not_rec_don]]
        table_non_abundant = [[not_abund_rec_don,abund_rec_don],[not_abund_not_rec_don,abund_not_rec_don]]

        try:
            odds_ratio = (abund_rec_don/(abund_rec_don+abund_not_rec_don))/(not_abund_rec_don/(not_abund_rec_don+not_abund_not_rec_don))
        except:
            odds_ratio = math.inf


        p_value = stats.barnard_exact(table,alternative="greater")

        table_abundance_rec_don_dict[compound] = {"table":table,"p_value":p_value.pvalue,"odds_ratio":odds_ratio}


        p_value_not = stats.barnard_exact(table_non_abundant,alternative="greater")

        table_not_abundant_rec_don_dict[compound]= {"table":table,"p_value":p_value_not.pvalue,"odds_ratio":odds_ratio}
        
    return table_abundance_rec_don_dict, table_not_abundant_rec_don_dict
        


In [142]:
def abundance_statistic_super_class(abundance_communities,receiver_or_donor="receiver"):
    
    
    table_abundance_rec_don_dict = {}
    table_not_abundant_rec_don_dict = {}

    for super_class in abundance_communities[abundance_communities.receiver_abundance_10].super_class.unique():
        if receiver_or_donor=="receiver":
            abund_rec_don = len(abundance_communities[(abundance_communities.receiver_abundance_10) & (abundance_communities.super_class==super_class)].receiver.unique())
            not_abund_rec_don = len(abundance_communities[(~abundance_communities.receiver_abundance_10) & (abundance_communities.super_class==super_class)].receiver.unique())
        elif receiver_or_donor=="donor":
            abund_rec_don = len(abundance_communities[(abundance_communities.donor_abundance_10) & (abundance_communities.super_class==super_class)].donor.unique())
            not_abund_rec_don = len(abundance_communities[(~abundance_communities.donor_abundance_10) & (abundance_communities.super_class==super_class)].donor.unique())
        
        
        abund_not_rec_don = len(high_abundance) - abund_rec_don
        not_abund_not_rec_don = len(low_abunance) - not_abund_rec_don


        table = [[abund_rec_don,not_abund_rec_don],[abund_not_rec_don,not_abund_not_rec_don]]
        table_non_abundant = [[not_abund_rec_don,abund_rec_don],[not_abund_not_rec_don,abund_not_rec_don]]

        try:
            odds_ratio = (abund_rec_don/(abund_rec_don+abund_not_rec_don))/(not_abund_rec_don/(not_abund_rec_don+not_abund_not_rec_don))
        except:
            odds_ratio = math.inf


        p_value = stats.barnard_exact(table,alternative="greater")

        table_abundance_rec_don_dict[super_class] = {"table":table,"p_value":p_value.pvalue,"odds_ratio":odds_ratio}


        p_value_not = stats.barnard_exact(table_non_abundant,alternative="greater")

        table_not_abundant_rec_don_dict[super_class]= {"table":table,"p_value":p_value_not.pvalue,"odds_ratio":odds_ratio}
        
    return table_abundance_rec_don_dict, table_not_abundant_rec_don_dict
        


**Assign abundant/not-abundant classifier**

In [143]:
abundance_communities = steadier_sample_cross.copy()
abundance_communities["donor_abundance_10"] = abundance_communities["donor"].map(lambda x: all_mags_paper.loc[x,"new_coverage"]>10)
abundance_communities["receiver_abundance_10"] = abundance_communities["receiver"].map(lambda x: all_mags_paper.loc[x,"new_coverage"]>10)

**Overview of abundant members**

In [144]:
high_abundance = set(list(abundance_communities[abundance_communities.donor_abundance_10].donor.unique()) + list(abundance_communities[abundance_communities.receiver_abundance_10].receiver.unique()))
low_abunance = set(list(abundance_communities[~abundance_communities.donor_abundance_10].donor.unique()) + list(abundance_communities[~abundance_communities.receiver_abundance_10].receiver.unique()))

#### Statistics for receiving

In [145]:
table_abundance_rec_dict, table_not_abundant_rec_dict = abundance_statistic(abundance_communities,receiver_or_donor="receiver")

**Abundant -compounds**

In [146]:
abund_rec_df = pd.DataFrame(table_abundance_rec_dict).T
abund_rec_df = statistics_adjustments(abund_rec_df)

abund_rec_df[(abund_rec_df.p_value_benjamini_h<0.05)&(abund_rec_df.odds_ratio>4.95)].sort_values("p_value")

,table,p_value,odds_ratio,i,p_value_benjamini_h


**Not abundant -compounds**

In [147]:
not_abund_rec_df = pd.DataFrame(table_not_abundant_rec_dict).T
not_abund_rec_df = statistics_adjustments(not_abund_rec_df)
not_abund_rec_df["odds_ratio"] = not_abund_rec_df["odds_ratio"].map(lambda x: 1/x)
not_abund_rec_df[(not_abund_rec_df.p_value_benjamini_h<0.05)& (not_abund_rec_df.odds_ratio>4.95)].sort_values("p_value")

,table,p_value,odds_ratio,i,p_value_benjamini_h


**SUPER CLASS**

In [148]:
table_abundance_rec_dict, table_not_abundant_rec_dict = abundance_statistic_super_class(abundance_communities,receiver_or_donor="receiver")

**Not abundant -superclass**

In [149]:
abund_rec_df = pd.DataFrame(table_abundance_rec_dict).T
abund_rec_df = statistics_adjustments(abund_rec_df)

abund_rec_df[(abund_rec_df.p_value_benjamini_h<0.05)& (abund_rec_df.odds_ratio>4.95)].sort_values("p_value")

,table,p_value,odds_ratio,i,p_value_benjamini_h


**Abundant -compounds**

In [150]:
not_abund_rec_df = pd.DataFrame(table_not_abundant_rec_dict).T
not_abund_rec_df = statistics_adjustments(not_abund_rec_df)
not_abund_rec_df["odds_ratio"] = not_abund_rec_df["odds_ratio"].map(lambda x: 1/x)
not_abund_rec_df[(not_abund_rec_df.p_value_benjamini_h<0.05)& (not_abund_rec_df.odds_ratio>4.95)].sort_values("p_value")

,table,p_value,odds_ratio,i,p_value_benjamini_h


#### Statistics for donating

In [151]:
table_abundant_donation_dict, table_not_abundant_donation_dict = abundance_statistic(abundance_communities,receiver_or_donor="donor")

**Abundant -compound**

In [152]:
abundance_don_df = pd.DataFrame(table_abundant_donation_dict).T
abundance_don_df = statistics_adjustments(abundance_don_df)
abundance_don_df[(abundance_don_df.p_value_benjamini_h<0.05) & (abundance_don_df.odds_ratio>4.95)].sort_values("p_value")


,table,p_value,odds_ratio,i,p_value_benjamini_h
L-Lysine,"[[6, 3], [3, 48]]",0.000079,11.333333,1.0,0.001580
Thymine C5H6N2O2,"[[5, 3], [4, 48]]",0.000807,9.444444,3.0,0.005377
Uracil,"[[5, 3], [4, 48]]",0.000807,9.444444,3.0,0.005377
Guanine,"[[4, 4], [5, 47]]",0.007336,5.666667,4.0,0.036681


**Not abundant -compound**

In [153]:
not_abundant_don_df = pd.DataFrame(table_not_abundant_donation_dict).T
not_abundant_don_df = statistics_adjustments(not_abundant_don_df)


not_abundant_don_df["odds_ratio"] = not_abundant_don_df["odds_ratio"].map(lambda x: math.inf if x==0 else 1/x)
not_abundant_don_df[(not_abundant_don_df.p_value_benjamini_h<0.05)& (not_abundant_don_df.odds_ratio>4.95)].sort_values("p_value")

,table,p_value,odds_ratio,i,p_value_benjamini_h


**SUPER CLASS**

In [154]:
table_abundant_donation_dict, table_not_abundant_donation_dict = abundance_statistic_super_class(abundance_communities,receiver_or_donor="donor")

**Abundant**

In [155]:
abundance_don_df = pd.DataFrame(table_abundant_donation_dict).T
abundance_don_df = statistics_adjustments(abundance_don_df)
abundance_don_df[(abundance_don_df.p_value_benjamini_h<0.05)& (abundance_don_df.odds_ratio>4.95)].sort_values("p_value")


,table,p_value,odds_ratio,i,p_value_benjamini_h


**Not abundant**

In [156]:
not_abundant_don_df = pd.DataFrame(table_not_abundant_donation_dict).T
not_abundant_don_df = statistics_adjustments(not_abundant_don_df)


not_abundant_don_df["odds_ratio"] = not_abundant_don_df["odds_ratio"].map(lambda x: math.inf if x==0 else 1/x)
not_abundant_don_df[(not_abundant_don_df.p_value_benjamini_h<0.05) & (not_abundant_don_df.odds_ratio>4.95)].sort_values("p_value")

,table,p_value,odds_ratio,i,p_value_benjamini_h


## Mass flow

#### Thymine C5H6N2O2

**Receiver of Thymine**

In [157]:
steadier_sample_cross[(steadier_sample_cross.compound=="Thymine C5H6N2O2")].value_counts("family_receiver")

family_receiver
f_Lachnospiraceae        8
Other                    7
f_CAG-74                 3
f_Sphaerochaetaceae      3
f_Desulfovibrionaceae    2
dtype: int64

In [158]:
steadier_sample_cross[(steadier_sample_cross.compound=="Thymine C5H6N2O2")].groupby(["family_receiver"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_receiver
f_Lachnospiraceae        0.000017
f_CAG-74                 0.000014
Other                    0.000005
f_Sphaerochaetaceae      0.000004
f_Desulfovibrionaceae    0.000003
Name: mass_rate*frequency, dtype: float64

**Donor of Thymine C5H6N2O2**

In [159]:
steadier_sample_cross[(steadier_sample_cross.compound=="Thymine C5H6N2O2")].value_counts("family_donor")

family_donor
Other                    10
f_Desulfovibrionaceae     8
f_Fibrobacteraceae        3
f_Lachnospiraceae         2
dtype: int64

In [160]:
steadier_sample_cross[(steadier_sample_cross.compound=="Thymine C5H6N2O2")].groupby(["family_donor"]).sum()["mass_rate*frequency"].sort_values(ascending=False)


family_donor
f_Desulfovibrionaceae    0.000027
Other                    0.000011
f_Lachnospiraceae        0.000003
f_Fibrobacteraceae       0.000002
Name: mass_rate*frequency, dtype: float64

In [161]:
steadier_sample_cross[(steadier_sample_cross.compound=="Thymine C5H6N2O2")].groupby(["community","family_receiver"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_receiver      
CD_A       f_Lachnospiraceae        3.334903e-06
CD_P       Other                    3.877135e-06
           f_Desulfovibrionaceae    3.036685e-06
           f_Lachnospiraceae        1.485775e-06
           f_Sphaerochaetaceae      5.755451e-07
CM_A       f_CAG-74                 1.136637e-06
CM_P       f_CAG-74                 1.245267e-05
           f_Sphaerochaetaceae      3.190788e-06
           Other                    7.530478e-07
M_P        f_Lachnospiraceae        1.236434e-05
           Other                    4.077591e-07
Name: mass_rate*frequency, dtype: float64

In [162]:
steadier_sample_cross[(steadier_sample_cross.compound=="Thymine C5H6N2O2")].groupby(["community","family_donor"]).sum().sort_values(["community","mass_rate*frequency"],ascending=[True,False])["mass_rate*frequency"]


community  family_donor         
CD_A       f_Desulfovibrionaceae    2.359785e-06
           f_Fibrobacteraceae       9.751177e-07
CD_P       Other                    8.975140e-06
CM_A       f_Fibrobacteraceae       1.136637e-06
CM_P       f_Desulfovibrionaceae    1.461857e-05
           Other                    1.777932e-06
M_P        f_Desulfovibrionaceae    1.023711e-05
           f_Lachnospiraceae        2.534990e-06
Name: mass_rate*frequency, dtype: float64